<a href="https://colab.research.google.com/github/niigoatnightcord/595project/blob/main/595hw4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
with open('mandelbrot.py', 'w') as f:
    f.write("""
import numpy as np
import matplotlib.pyplot as plt

#generate complex number
x, y = np.mgrid[-2:1:1000j, -1.5:1.5:1000j]
c = x + 1j * y

#set threshhold of divergence and max iteration
threshold = 50
max_iter = 100

#initialize z with the same shape of c and set all values to zero
z = np.zeros_like(c, dtype = complex)

#decide whether a point is in the Mandelbrot set
for n in range(max_iter):
    mask = np.abs(z) < threshold #mask select
    z[mask] = z[mask] ** 2 + c[mask]

plt.imshow(mask.T, extent=[-2, 1, -1.5, 1.5], cmap='gray') # arry to image needs transpose (mask.T)
plt.colorbar()
plt.savefig('mandelbrot.png', dpi=150)
plt.grid(True)
plt.show()
""")

In [2]:
with open('markov chain.py', 'w') as f:
    f.write("""

import numpy as np

#construct a random 5x5 transition matrix P and normalize each row
P = np.random.rand(5, 5)
P = P / P.sum(axis=1, keepdims=True)  #normalize each row to sum to 1
# axis = 1 is row, axis = 0 is column

# construct a random probability vector p and normalize it
p = np.random.rand(5)
p = p / p.sum()  #normalize so that sum(p) = 1

#apply the transition rule 50 times to obtain p50
p_current = p
for _ in range(50):
    p_current = np.dot(P.T, p_current)

p_50 = p_current

# eigenvector corresponding to the eigenvalue 1 for P.T (stationary distribution)
eigenvalues, eigenvectors = np.linalg.eig(P.T)

#index of the eigenvalue closest to 1
index = np.argmin(np.abs(eigenvalues - 1))

v = eigenvectors[:, index].real  #eigenvector associated with eigenvalue 1 and take the real part
v = v / v.sum()  #normalize v to make it a probability vector (stationary distribution)

#component-wise difference between p50 and the stationary distribution
difference = np.abs(p_50 - v)

print("Transition matrix P:\n", P)
print("\nInitial probability vector p:\n", p)
print("\nProbability vector after 50 transitions (p50):\n", p_50)
print("Eigenvalue closest to 1:\n", eigenvalues[index])
print("\nStationary distribution (scaled eigenvector):\n", v)
print("\nComponent-wise difference between p50 and stationary distribution:\n", difference)
print("\nDo p50 and the stationary distribution match within 10^-5?", np.all(difference < 1e-5))
""")

In [3]:
with open('taylor.py', 'w') as f:
    f.write("""
import numpy as np
import pandas as pd
import sympy as sp
import time
import matplotlib.pyplot as plt

#calculate Taylor series approximation of given func
def taylor_series(func, start, end, degree, fixed_c):
    x = sp.symbols('x')#symbolic variation for differentiation
    func_sym = func(x)#sybolic function
    derivatives = [func_sym]#first term is func
    for i in range(1, degree + 1): #from 1 to degree
        derivatives.append(sp.diff(derivatives[i - 1], x)) #append ith derivative

    x_values = np.linspace(start, end, 100)
    #generate an array of x in the interval of [start,end] of 100 pts
    y_approx = np.zeros_like(x_values)#initialize an array for storage

    #compute approx for each xi
    for i, xi in enumerate(x_values):
        approx_val = 0
        for n in range(degree + 1):
            derivative_val = derivatives[n].subs(x, fixed_c)#value of nth derivative at x=c
            approx_val += (derivative_val / np.math.factorial(n)) * ((xi - fixed_c) ** n)
            #nth term of approx and add to the formula
        y_approx[i] = approx_val #store the computed approx

    return x_values, y_approx

# evaluate Taylor series with varying degrees
def evaluate_varying_degrees(func, start, end, fixed_c, initial_degree, final_degree, degree_step):
    x = sp.symbols('x')#symbolic var x
    func_sym = func(x)#symbolic func
    f = sp.lambdify(x, func_sym, 'numpy')#convert symbolic fuc to numpy func

    x_values = np.linspace(start, end, 100)
    y_true = f(x_values)#value of true func

    # DataFrame to store results for each degree
    results = {'Degree': [], 'Difference': [], 'Time Taken': []}

    #loop through certain range
    for degree in range(initial_degree, final_degree + 1, degree_step):
        tic = time.time()
        _, y_approx = taylor_series(func, start, end, degree, fixed_c)
        toc = time.time()

        #diiference btw approx and true func
        difference = np.sum(np.abs(y_true - y_approx))
        time_taken = toc - tic

        #store the results in DataFrame
        results['Degree'].append(degree)
        results['Difference'].append(difference)
        results['Time Taken'].append(time_taken)

    #convert results dictionary to DataFrame and save as CSV
    df_results = pd.DataFrame(results)
    df_results.to_csv('taylor_values.csv', index=False)

if __name__ == "__main__":
    #define the function by anonymous func
    func = lambda x: x * sp.sin(x)**2 + sp.cos(x)

    #plot Taylor series approximation for a specific degree
    x_values, y_approx = taylor_series(func, -10, 10, 99, 0)
    f = sp.lambdify(sp.symbols('x'), func(sp.symbols('x')), 'numpy')
    y_true = f(x_values)

    #plot the function and approximation
    plt.plot(x_values, y_true, label='f(x) = x * sin^2(x) + cos(x)', color='blue')
    plt.plot(x_values, y_approx, label='Taylor Approximation (m=99)', color='orange', linestyle='--')
    plt.xlabel('x')
    plt.ylabel('f(x)')
    plt.legend()
    plt.title('Taylor Series Approximation of f(x)')
    plt.show()

    #evaluate Taylor series with varying degrees and save results
    evaluate_varying_degrees(func, start=-10, end=10, fixed_c=0, initial_degree=50, final_degree=100, degree_step=10)
""")

In [4]:
from google.colab import files

files.download('mandelbrot.py')
files.download('markov chain.py')
files.download('taylor.py')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [8]:
from google.colab import files
files.download('taylor_values.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>